In [51]:
import pandas as pd
import plotly as px
import numpy as np

In [52]:
def scelta_pattern(media_annuale,media_mensile, media_bimestre, media_trimestre, media_semestre, array_emissioni,num,df_final,mese_start,giorno_cambio_mese):
    #array_emissioni=array dei consumi
    array_pattern = []
    # codice che sceglie pattern     
    prec=0
    val=0

    #Media mensile
    media_corto=media_mensile[mese_start-1]
    #Media trimestre
    if mese_start<=3: media_lungo=media_trimestre[0]
    elif mese_start<=6: media_lungo=media_trimestre[1]
    elif mese_start<=9: media_lungo=media_trimestre[2]
    else: media_lungo=media_trimestre[3]
    #Media bimestre
    #if mese_start<=2: media_lungo=media_bimestre[1]
    #elif mese_start<=4: media_lungo=media_bimestre[2]
    #elif mese_start<=6: media_lungo=media_bimestre[3]
    #elif mese_start<=8: media_lungo=media_bimestre[4]
    #elif mese_start<=10: media_lungo=media_bimestre[5]
    #else: media_lungo=media_bimestre[6]
    #Media semestre
    #if mese_start<=6: media_lungo=media_semestre[1]
    #else: media_lungo=media_semestre[2]


    for i in range(len(array_emissioni) - 1):
        if giorno_cambio_mese==i:
            #Va direttamente al mese successivo
            media_corto=media_mensile[mese_start]
            #Controlla se cambia trimestre
            if (mese_start+1)<=3: media_lungo=media_trimestre[0]
            elif (mese_start+1)<=6: media_lungo=media_trimestre[1]
            elif (mese_start+1)<=9: media_lungo=media_trimestre[2]
            else: media_lungo=media_trimestre[3]
            
        pattern_day=""

        #PATTERN MEDIA CORTO
        if array_emissioni[i]<(media_corto/4):
            pattern_day+= "cA"
        elif array_emissioni[i]<(media_corto/2):
            pattern_day+= "cB"
        elif array_emissioni[i]<(media_corto*(3/4)):
            pattern_day+= "cC"
        elif array_emissioni[i]<media_corto:
            pattern_day+= "cD"
        elif array_emissioni[i]<(media_corto*(5/4)):
            pattern_day+= "CA"
        elif array_emissioni[i]<(media_corto*(3/2)):
            pattern_day+= "CB"
        elif array_emissioni[i]<(media_corto*(7/4)):
            pattern_day+= "CC"
        else:
            pattern_day+= "CD"
        #PATTERN MEDIA LUNGO
        if array_emissioni[i]<(media_lungo / 4):
            pattern_day += "lA"
        elif array_emissioni[i]<(media_lungo / 2):
            pattern_day += "lB"
        elif array_emissioni[i]<(media_lungo * (3 / 4)):
            pattern_day += "lC"
        elif array_emissioni[i]<media_lungo:
            pattern_day += "lD"
        elif array_emissioni[i]<(media_lungo * (5 / 4)):
            pattern_day += "LA"
        elif array_emissioni[i]<(media_lungo * (3 / 2)):
            pattern_day += "LB"
        elif array_emissioni[i]<(media_lungo * (7 / 4)):
            pattern_day += "LC"
        else:
            pattern_day += "LD"

        #CALCOLARE RAPPORTO TRA LE DUE EMISSIONI
        if i>0:
            if array_emissioni[i-1] != 0:
                val = array_emissioni[i] / array_emissioni[i-1]
            else:
                val = array_emissioni[i]
            prec=array_emissioni[i-1]
        else: val = array_emissioni[i]

        #PATTERN RAPPORTO
        if val == 0:  # zero emissioni
            if prec == 0:#solitamente il LUN
                pattern_day+= "0A"
            else:
                pattern_day+= "0B"

        elif val <= 1:  # emissioni diminuite
            if val < 0.20:
                pattern_day+= "1A"
            elif val < 0.50:
                pattern_day+= "1B"
            elif val < 0.75:
                pattern_day+= "1C"
            else:
                pattern_day+= "1D"

        elif val <= 2:  # emissioni aumentate leggermente
            if val < 1.20:
                pattern_day+= "2A"
            elif val < 1.50:
                pattern_day+= "2B"
            elif val < 1.75:
                pattern_day+= "2C"
            else:
                pattern_day+= "2D"

        elif val <= 10:  # emissioni aumentate
            if val < 7:
                pattern_day+= "3A"
            else: pattern_day+= "3B" 

        else:
            if i>0: pattern_day+= "4B" #valore un po strano
            else: pattern_day+= "4A" # possibile, probabilmente LUN
        array_pattern.append(pattern_day)
        
    # Stampa l'array
    print("\nSettimana numero: "+str(num))
    print("\nArray con i pattern della settimana da LUN a DOM")
    print(array_pattern)
    #print("\nTabella e Plot di df_final")
    #display(df_final)
    #df_final.plot()

In [53]:
def ciclo_pattern_settimane_precise(df_daily,media_annuale, media_mensile, media_bimestre, media_trimestre, media_semestre):
    num=0 #numero settimana
    start_date = pd.Timestamp('2019-01-01')
    end_date = pd.Timestamp('2019-12-31')
    #trovo il prossimo lunedi
    start_date += pd.DateOffset(days=(7 - start_date.weekday())) #0=lunedi 6=domenica
    current_date=start_date
    while current_date <= end_date:
        weekstart_date=current_date
        weekend_date=weekstart_date + pd.DateOffset(days=6)
        if weekend_date > end_date:
            break
        # Creare un range di date
        date_rng = pd.date_range(start=weekstart_date, end=weekend_date, freq='D')
        current_date += pd.DateOffset(weeks=1)

        # Creare un DataFrame vuoto con il range di date specificato
        df_final = pd.DataFrame(date_rng, columns=['date'])
        df_final.set_index('date', inplace=True)
        
        # Unire il DataFrame giornaliero con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
        df_final = df_final.join(df_daily, how='left').fillna(0)
        
        array_emissioni = df_final['consumption'].to_numpy()

        # Controlliamo se ci troviamo in una settimana tra due mesi, causa controlli con le medie
        mese_start = weekstart_date.month
        giorno_cambio_mese = -1
        for i in range(7):
            current_day = weekstart_date + pd.DateOffset(days=i)
            if current_day.month != mese_start:
                giorno_cambio_mese = i #gia sappiamo che se cambia mese aumenta mese_start di 1
                break

        scelta_pattern(media_annuale,media_mensile, media_bimestre, media_trimestre, media_semestre, array_emissioni,num,df_final,mese_start,giorno_cambio_mese)
        num=num+1

In [54]:
def ciclo_pattern(df_daily,media_annuale,array_mensile,array_bimestre,array_trimestre,array_semestre):
    num=0 #numero settimane
    start_date = pd.Timestamp('2019-01-01')
    current_date=start_date
    end_date = pd.Timestamp('2019-12-31')
    #trovo il prossimo lunedi
    start_date += pd.DateOffset(days=(7 - start_date.weekday())) #0=lunedi 6=domenica
    while current_date <= end_date:
        weekstart_date=current_date
        weekend_date=weekstart_date + pd.DateOffset(days=6)
        if weekend_date > end_date:
            weekend_date = end_date
        # Creare un range di date
        if num==0: #Prima settimana variabile poiche non inizia con lunedi l'anno
            date_rng = pd.date_range(start=weekstart_date, end=start_date-pd.DateOffset(days=1) , freq='D')    
            current_date =start_date
        else:
            date_rng = pd.date_range(start=weekstart_date, end='2019-02-24', freq='D')
            current_date += pd.DateOffset(weeks=1)

        # Creare un DataFrame vuoto con il range di date specificato
        df_final = pd.DataFrame(date_rng, columns=['date'])
        df_final.set_index('date', inplace=True)
        
        # Unire il DataFrame giornaliero con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
        df_final = df_final.join(df_daily, how='left').fillna(0)
        
        array_emissioni = df_final['consumption'].to_numpy()

        scelta_pattern(media_annuale,array_mensile,array_bimestre,array_trimestre,array_semestre, array_emissioni,num,df_final)
        num=num+1


In [55]:
def calcola_media_semestrale(df_final):
    periodi_semestrali = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-12-31'))
    ]
    media_semestrale = []
    for start, end in periodi_semestrali:
        media = df_final.loc[start:end].mean()
        media_semestrale.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_semestrale
    

In [56]:
def calcola_media_trimestrale(df):
    periodi_trimestrali = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-03-31')),
        (pd.Timestamp('2019-04-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-09-30')),
        (pd.Timestamp('2019-10-01'), pd.Timestamp('2019-12-31'))
    ]

    # Calcolare la media per ciascun periodo
    media_trimestrale = []
    for start, end in periodi_trimestrali:
        media = df.loc[start:end].mean()
        media_trimestrale.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_trimestrale

In [57]:
def calcola_media_bimestrale(df):
    periodi_bimestrali = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-02-28')),
        (pd.Timestamp('2019-03-01'), pd.Timestamp('2019-04-30')),
        (pd.Timestamp('2019-05-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-08-31')),
        (pd.Timestamp('2019-09-01'), pd.Timestamp('2019-10-31')),
        (pd.Timestamp('2019-11-01'), pd.Timestamp('2019-12-31'))
    ]

    # Calcolare la media per ciascun periodo
    media_bimestrale = []
    for start, end in periodi_bimestrali:
        media = df.loc[start:end].mean()
        media_bimestrale.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_bimestrale

In [58]:
def calcola_media_mensile(df):
    periodi_mensili = [
        (pd.Timestamp('2019-01-01'), pd.Timestamp('2019-01-31')),
        (pd.Timestamp('2019-02-01'), pd.Timestamp('2019-02-28')),
        (pd.Timestamp('2019-03-01'), pd.Timestamp('2019-03-31')),
        (pd.Timestamp('2019-04-01'), pd.Timestamp('2019-04-30')),
        (pd.Timestamp('2019-05-01'), pd.Timestamp('2019-05-31')),
        (pd.Timestamp('2019-06-01'), pd.Timestamp('2019-06-30')),
        (pd.Timestamp('2019-07-01'), pd.Timestamp('2019-07-31')),
        (pd.Timestamp('2019-08-01'), pd.Timestamp('2019-08-31')),
        (pd.Timestamp('2019-09-01'), pd.Timestamp('2019-09-30')),
        (pd.Timestamp('2019-10-01'), pd.Timestamp('2019-10-31')),
        (pd.Timestamp('2019-11-01'), pd.Timestamp('2019-11-30')),
        (pd.Timestamp('2019-12-01'), pd.Timestamp('2019-12-31'))
    ]

    # Calcolare la media per ciascun periodo
    media_mensile = []
    for start, end in periodi_mensili:
        media = df.loc[start:end].mean()
        media_mensile.append(media.values[0])  # Assumendo che ci sia una sola colonna di valori

    return media_mensile

In [59]:
def read_fixture(fixture: str, rule='T'):
    #Legge CSV
    df = pd.read_csv(f'./data/feed_{fixture}.MYD.csv', delimiter=' ', header=None, names=['date', fixture])
    df['date'] = pd.to_datetime(df['date'], unit='s')
    
    # Rinomina la colonna fixture in 'consumption'
    df.rename(columns={fixture: 'consumption'}, inplace=True)
    df.set_index('date', inplace=True)
    
    # Resample il DataFrame usando la frequenza specificata (minutale 'T')
    df_minutely = df.resample(rule).sum()
    
    # Ora raggruppiamo i dati minutali in giornate
    df_daily = df_minutely.resample('D').sum()

    # Creare un DataFrame vuoto con il range di date specificato
    date_rng = pd.date_range(start='2019-01-01', end='2019-12-31' , freq='D')
    df_final = pd.DataFrame(date_rng, columns=['date'])
    df_final.set_index('date', inplace=True)
    
    # Unire il DataFrame giornaliero con il DataFrame vuoto per assicurarsi che tutte le date siano presenti
    df_final = df_final.join(df_daily, how='left').fillna(0)
    
    # Media annuale
    media_annuale = df_final['consumption'].mean()
    # Media per ogni mese
    media_mensile = calcola_media_mensile(df_final)
    # Media per ogni bimestre (2 mesi)
    media_bimestre= calcola_media_bimestrale(df_final)
    # Media per ogni trimestre (3 mesi)
    media_trimestre = calcola_media_trimestrale(df_final)
    # Media per ogni semestre (6 mesi)
    media_semestre = calcola_media_semestrale(df_final)

    #debug medie
    stampa_medie(media_annuale, media_mensile, media_bimestre, media_trimestre, media_semestre)
    
    ciclo_pattern_settimane_precise(df_final,media_annuale, media_mensile, media_bimestre, media_trimestre, media_semestre)


In [60]:
def stampa_medie(media_annuale, media_mensile, media_bimestre, media_trimestre, media_semestre):
    print("Media annuale: ") 
    print(media_annuale)

    print("\nMedie mensili:") 
    print(media_mensile)

    print("\nMedie bimestrali:")
    print(media_bimestre)

    print("\nMedie trimestrali:")
    print(media_trimestre)

    print("\nMedie semestrali:")
    print(media_semestre)

In [61]:
def range_plot(df):
    fig = px.line(df, y='consumption', labels={'consumption': 'Volume [L]'})
    fig.update_xaxes()
    fig.update_yaxes(fixedrange=False)
    fig.update_layout(font=dict(size=22))#grandezza font
    fig.show()

In [62]:
#Nome dispositivo, tipi medie: Mensile(M),Bimestre(B),Trimestre(T),Semestre(S),Annuale(A) 
#TO DO Scelta della media direttamente da qua, inserendo due medie o tre 
#media breve=mensile, bimestre, trimestre 
#media lunga=trimestre,semestre, annuale
read_fixture('Washbasin')

Media annuale: 
5679.882191780822

Medie mensili:
[0.0, 5361.964285714285, 6614.870967741936, 8858.8, 7666.451612903225, 13478.5, 6155.0, 2841.5806451612902, 7151.266666666666, 10217.741935483871, 0.0, 0.0]

Medie bimestrali:
[2544.6610169491523, 7718.44262295082, 10524.83606557377, 4498.290322580645, 8709.639344262296, 0.0]

Medie trimestrali:
[3946.6222222222223, 9975.593406593407, 5363.391304347826, 3442.9347826086955]

Medie semestrali:
[6977.762430939227, 4403.163043478261]

Settimana numero: 0

Array con i pattern della settimana da LUN a DOM
['CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A']

Settimana numero: 1

Array con i pattern della settimana da LUN a DOM
['CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A']

Settimana numero: 2

Array con i pattern della settimana da LUN a DOM
['CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A']

Settimana numero: 3

Array con i pattern della settimana da LUN a DOM
['CDlA0A', 'CDlA0A', 'CDlA0A', 'CDlA0A', 'cAlA0A'